# Tutorial 3: Fixed dimension MCMC with Eryn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from lisatools.utils.constants import *
from copy import deepcopy  # can be useful

In the third tutorial, we are going to learn how to use MCMC through `Eryn` to do our data analysis investigations. In this tutorial, we will stick with simple examples and simple signals (like in Tutorial 1). In later tutorials, we will use eryn with real GW signals. 

## Task 1: build your own basic MCMC

In order to better understand MCMC and `Eryn`, we are going to start by building our own MCMC algorithm for a simple problem. We will do this with a single-dimensional Gaussian likelihood centered on ($\mu=0$) and a unit standard deviation ($\sigma=1$) and a uniform prior. 

In [ ]:
def log_like_gauss(x):
    return -0.5 * x ** 2 - 1/2 * np.log(2 * np.pi * 1.0) # 1.0 is sigma

In [ ]:
x_vals = np.linspace(-10.0, 10.0, 1000)
# notice the `exp` in there because we are working with the log of the likelihood.
plt.plot(x_vals, np.exp(log_like_gauss(x_vals)))
plt.axvline(0.0)

We are just going to write this into a simple loop. Gather your samples into the chain list.

In [ ]:
num_steps = 100000

# get random starting point
current_point = np.random.uniform(-10.0, 10.0)
current_likelihood = log_like_gauss(current_point)
chain = []
for step in range(num_steps):
    # propose new point using a Gaussian distribution with standard deviation of 0.5
    # hint: use current + sigma * N() where N() is a draw from a normal distribution 
    # (np.random.randn)

    
    # get new likelihood

    
    # calculate the change in posterior
    # here we are using a uniform prior, so its value will not change, so for now do not
    # worry about the prior.
    
    
    # accept or reject
    # if change in log posterior is greater than log(np.random.rand()), accept, else reject
    
    
    if accept:
        # update info if accepted

    # append point to chain
    

Now plot the chain. You will notice that the beginnning requires a "burn in" phase. So, when calculating the posterior, you must remove this. There are ways to calculate how much burn in is appropriate, but we will not get into that here. 

Plot the posterior distribution generated by the MCMC and compare it against the true Likelihood plot from above. Remember to set `density=True` when building the histogram.

If we take a step back here, we realize MCMC is really just a way to draw samples from a distribution. When the distribution is simple like this example, it will usually be available already with no need to run MCMC (like in scipy). However, in our area of work, the Likelihood distribution tends to be very difficult to deal with, which is why MCMC is so useful for us. To illustrate this, we will draw samples from the same distribution available in Numpy (`np.random.randn`).

## Task 2: Use Eryn to reproduce the above result.

Now use the `EnsembleSampler` in `Eryn` to reproduce the above results. If you are struggling, look at the `Eryn` tutorial. For Eryn, you will need to define a prior. Use `uniform_dist` to create a simple uniform distribution. For this example, you only need to worry about the Ensemble Sampler keyword arguments. Leave those all as default values and only enter the arguments.

Useful documentation:
* [EnsembleSampler](https://mikekatz04.github.io/Eryn/html/user/ensemble.html#eryn.ensemble.EnsembleSampler)
* [State](https://mikekatz04.github.io/Eryn/html/user/state.html#eryn.state.State)
* [uniform_dist](https://mikekatz04.github.io/Eryn/html/user/prior.html#eryn.prior.uniform_dist)
* [ProbDistContainer](https://mikekatz04.github.io/Eryn/html/user/prior.html#eryn.prior.ProbDistContainer)
* [Backend](https://mikekatz04.github.io/Eryn/html/user/backend.html#eryn.backends.Backend)

In [ ]:
# imports
from eryn.ensemble import EnsembleSampler
from eryn.state import State
from eryn.prior import uniform_dist, ProbDistContainer

Initialize your sampler here. The prior initialization should take the form: `ProbDistContainer({0: prior distribution})`.

Now choose your starting point (one for each walker). You can draw from the prior or not. Just make sure to remove any burn in at the end. Then put the starting point into a `State` object. The key here is the start point should have shape `(1, nwalkers, 1, ndim)`. Here the two 1s are for temperatures (which we are not using right now) and the number of leaves or model count (this is for RJ). After this is complete, run the sampler.

Documentation:
* [run_mcmc](https://mikekatz04.github.io/Eryn/html/user/ensemble.html#eryn.ensemble.EnsembleSampler.run_mcmc)

Examine the output chains by using the backend:

* [Backend](https://mikekatz04.github.io/Eryn/html/user/backend.html#eryn.backends.Backend)

Compare your output chains to the injected Gaussian distribution. They should match. Make sure when you plot the histogram, you set `density=True`. 

## Task 3: Parallel Tempering

Now we will add parallel tempering. In order to examine the effect of addering tempering, we are going to look at a 1D distribution with two Gaussian peaks with different weights. We have provided the log Likelihood function for this exercise. You just set the x limits to proper normalize the distribution.

Parallel tempering works by supressing the Log Likelihood in comparison to the log prior: `1/T * logL + logp`. This effect lowers the peaks making it easier for higher temperature chains to traverse a low-Likelihood portion of the Likelihood surface. This helps to properly sample distributions with multiple posterior modes. 

We will start by sampling without tempering to see how well that does. Then we will add tempering to see the improvement. 

In [ ]:
from scipy.special import logsumexp

class LogLikeTwoGuass:
    def __init__(self, x_min: float, x_max: float):
        num_for_norm = 100000
        self.norm = 1.0
        x_vals = np.linspace(x_min, x_max, num_for_norm)
        y_vals = np.array([np.exp(self.log_like_two_gauss(x_tmp)) for x_tmp in x_vals])
        self.norm = np.trapz(y_vals, x=x_vals)
        
    def log_like_two_gauss(self, x):
        return logsumexp(np.array([(np.log(0.2) + -0.5 * (x - 50.0) ** 2), (np.log(0.8) + -0.5 * (x + 50.0) ** 2)]), axis=0) - np.log(self.norm)

In [ ]:
like = LogLikeTwoGuass(-1000.0, 1000.0)

In [ ]:
x_vals = np.linspace(-100.0, 100.0, 1000)
# notice the `exp` in there because we are working with the log of the likelihood.
plt.plot(x_vals, np.exp(like.log_like_two_gauss(x_vals)))
#plt.plot(x_vals, np.exp(np.array([like.log_like_two_gauss(x_tmp) for x_tmp in x_vals])))


Start with a non-tempered sampler. Run it and plot the histogram over the injected Likelihood. 

### Question

The plot should not look correct. Why do you think this is? When not using tempering, what determines the number of walkers inhabiting each peak?

Now add temperatures by providing `tempering_kwargs` kwarg to `EnsembleSampler.

Useful Documentation:
* [EnsembleSampler](https://mikekatz04.github.io/Eryn/html/user/ensemble.html#eryn.ensemble.EnsembleSampler)
* [TemperatureControl](https://mikekatz04.github.io/Eryn/html/user/temper.html#eryn.moves.tempering.TemperatureControl)

Sample start points again from the prior with shape `(ntemps, nwalkers, 1, ndim)`. Then run the sampler.

Plot the chains over the injection distribution.

Now the plots should match very weel. Can you describe the change that took place?

## Task 6: Add GWs!

Now we will add GW signals in the form of the simple Sinusoid we used in the first tutorial. We will start with the waveform function. Set up the `DataResidualArray` (inject whatever parameter you would like), `SensitivityMatrix`, and `AnalysisContainer`. Remember, we do not have a response on this signal, so the sensitivity curve should be `LISASens`. 

In [ ]:
from lisatools.datacontainer import DataResidualArray
from lisatools.analysiscontainer import AnalysisContainer
from lisatools.sensitivity import SensitivityMatrix, LISASens

We will add the waveform for you.

In [ ]:
def sinusoidal_waveform(A: float, f0: float, phi0: float, t: np.ndarray, **kwargs) -> [np.ndarray, np.ndarray]:
    h1 = A * np.sin(2 * np.pi * (f0 * t) + phi0)
    h2 = A * np.cos(2 * np.pi * (f0 * t) + phi0)
    return [h1, h2]

Generate the injection data, input it into a `DataResidualArray`, load a `SensitivityMatrix`, and store everything in an `AnalysisContainer`, including the sinusoidal waveform generator. 

Calculate the SNR of the injection. (`AnalysisContainer.calculate_signal_snr`)

Biuld the prior distributions for the three parameters.

Initialize the sampler. For the Likelihood function, you can use `AnalysisContainer.eryn_likelihood_function`. For  now, we will not use tempering. This generation scheme is not efficient in terms of the Likelihood evaluation time. So, we will just run this as a quick example. You can run it longer later on and/or parallelize it with Eryn's `pool` capabilities. This basic example we are using illustrates the need for fast waveform generation capabilities. 

Generate the start state and run the sampler. 

You can examine the chains and/or Likelihood values if you want, but you will likely have to run it for longer to get any reasonable results. 

### Question

What types of things can we do to speed up this calculation? Translate this into the waveform environment. 

## Task 4: Calculate the evidence

Calculate the evidence for the single-peaked Gaussian distribution using thermodynamic integration from Eryn. Use 10 walkers and 50 temperatures to start. You can play with these numbers later and observe their effect on the measurement. Use the `burn` kwarg when running `run_mcmc`. **Important**: for the `tempering_kwargs` keyword argument for `EnsembleSampler`, you must add to the dictionary: `stop_adaptation=burn`. This will adapt the temperatures during burn in and then hold them fixed while recording samples. 

Useful documentation:
* [thermodynamic_integration_log_evidence](https://mikekatz04.github.io/Eryn/html/user/utils.html#eryn.utils.utility.thermodynamic_integration_log_evidence)

In [ ]:
# imports
from eryn.utils.utility import thermodynamic_integration_log_evidence

Initialize the sampler. Run the MCMC.

1) Compute the average likelihood across all samples and walkers **within** each temperature chain.
2) Get the inverse temperatures (`betas`) out of the sampler backend and make sure they are the same over the entire run.
3) Calculate the evidence.

Useful Documentation:
* [thermodynamic_integration_log_evidence](https://mikekatz04.github.io/Eryn/html/user/utils.html#eryn.utils.utility.thermodynamic_integration_log_evidence)
* [Backend.get_log_like](https://mikekatz04.github.io/Eryn/html/user/backend.html#eryn.backends.Backend.get_log_like)
* [Backend.get_betas](https://mikekatz04.github.io/Eryn/html/user/backend.html#eryn.backends.Backend.get_betas)

## Task 5: Model Selection

Perform model comparison between a Gaussian and Cauchy pulse using thermodynamic integration of the evidence.

Here are the pulse functions and the Likelhiood. We will assume each pulse has an amplitude and mean parameter. We set the standard deviation to be 1. 

In [ ]:
from scipy.stats import cauchy
def gaussian_pulse(x, a, b):
    f_x = a * np.exp(-((x - b) ** 2) / (2 * 1.0 ** 2))
    return f_x

def cauchy_pulse(x, a, b):
    f_x = a * cauchy.pdf(x - b)
    return f_x

def log_like_fn(params, t, data, sigma, which_template):

    pulse_gen = gaussian_pulse if which_template == "gauss" else cauchy_pulse
    template = pulse_gen(t, *params)

    ll = -0.5 * np.sum(((template - data) / sigma) ** 2, axis=-1)
    return ll

In [ ]:
t_vals = np.linspace(-10.0, 10.0, 1000)
sigma = 0.2
amp_true = 4.0
mean_true = 0.0
true_data = gaussian_pulse(t_vals, amp_true, mean_true)
data = true_data + np.random.randn(*t_vals.shape) * sigma
cauchy_data = cauchy_pulse(t_vals, amp_true * 3, mean_true)
plt.plot(t_vals, data, label="data")
plt.plot(t_vals, true_data, label="gauss")
plt.plot(t_vals, cauchy_data, label="cauchy")
plt.legend()
# plt.plot(x_vals, np.exp(log_like_fn()))
# plt.plot(x_vals, np.exp(log_like_gauss(x_vals)))

We need to run separate sampler objects for each model. 

The priors are identical really for the two models. The amplitude prior should span the injection values. The mean prior should span the domain of time.

Initialize the priors for each, the sampler for each, sample a start point for each, and run both samplers with the setup we used above. 

Repeat the calculation from above for both models. Then find Bayes Factor.

### Question

If we change the noise, what effect will this have on our results?